In [9]:
# imports:

import pandas as pd
from datetime import datetime
import time
import seaborn as sns
import datapane as dp
import altair as alt


## Question 1

You are provided with data with 10-mins, 60-mins and 1-day resolution (Merge.csv)
Please merge them into a pandas Dataframe with 2-hours resolution in between 7:00 – 17:00 only as index.
Please take the average of the 10-mins and 60-mins resolution prices during the 2-hours window and forward fill the 1-day resolution prices in the 2-hours window.

In [7]:
merge = pd.read_csv("./Data analyst coding challenge/Merge.csv")
merge["Datetime"]= pd.to_datetime(merge["Datetime"], infer_datetime_format=True)
merge

,Resolution,Datetime,Price
0,10MIN,2021-11-01 07:00:00,70.000
1,10MIN,2021-11-01 07:10:00,NaN
2,10MIN,2021-11-01 07:20:00,71.500
3,10MIN,2021-11-01 07:30:00,71.375
4,10MIN,2021-11-01 07:40:00,NaN
...,...,...,...
1565,D,2021-11-26 00:00:00,88.350
1566,D,2021-11-27 00:00:00,NaN
1567,D,2021-11-28 00:00:00,NaN
1568,D,2021-11-29 00:00:00,94.900


In [8]:
two_hour = merge.groupby([pd.Grouper(key="Datetime", freq="2H", origin="07:00:00")]).mean().reset_index().set_index("Datetime")
two_hour

,Price
Datetime,
2021-10-31 23:00:00,66.100000
2021-11-01 01:00:00,NaN
2021-11-01 03:00:00,NaN
2021-11-01 05:00:00,NaN
2021-11-01 07:00:00,70.925000
...,...
2021-11-30 07:00:00,97.455556
2021-11-30 09:00:00,98.826786
2021-11-30 11:00:00,96.619643


In [6]:
# I still need to forward fill the dates with the resolution being "D"
final = two_hour.between_time("7:00", "17:00")
final

,Price
Datetime,
2021-11-01 07:00:00,70.925000
2021-11-01 09:00:00,71.494643
2021-11-01 11:00:00,68.635714
2021-11-01 13:00:00,63.314286
2021-11-01 15:00:00,65.630357
...,...
2021-11-30 07:00:00,97.455556
2021-11-30 09:00:00,98.826786
2021-11-30 11:00:00,96.619643


## Question 2

You are provided with a daily energy consumption data from 2016 to date (Consumption.csv).
Please create a Pandas DataFrame with to show the consumption of each year. The expected format is to have the year number as column name and mm-dd as index.
Please also create a seasonal plot showing 5-years (2016-2020) range (shaded) & average (dashed line), and year 2021 (line) & 2022 (line).
Please comment on your observation on the plot

In [10]:
# Load data
consumption = pd.read_csv("./Data analyst coding challenge/Consumption.csv")
time.sleep(1)
print(consumption)
# this throws an error because date formats in the rows are different
print("Converting date column into datetype object")
try:
    consumption["Date"]= pd.to_datetime(consumption["Date"], infer_datetime_format=True)
except:
    print("WARNING: there are some different date formats in the rows, so can't infer date format")
 # inspect for missing values:
print("--- examining missing values:---")
time.sleep(.5)
print(f"There are: \n {consumption.isna().sum()} \n values missing values across the consumption dataframe rows")
 # No values are missing across dataframe rows


            Date  Consumption
0     01/01/2016     276.2910
1     02/01/2016     294.6780
2     03/01/2016     306.3096
3     04/01/2016     370.0566
4     05/01/2016     367.3998
...          ...          ...
2463  29/09/2022     258.0822
2464  30/09/2022     242.5950
2465  01/10/2022     188.0658
2466  02/10/2022     190.6092
2467  03/10/2022     210.3246

[2468 rows x 2 columns]
Converting date column into datetype object
--- examining missing values:---
There are: 
 Date           0
Consumption    0
dtype: int64 
 values missing values across the consumption dataframe rows


In [11]:
# check for dupes:
# across duplicates
time.sleep(.5)
consumption.duplicated().sum() # there are zero across duplicates
# date duplicates:
consumption["Date"].duplicated().sum() 

consumption["Date"].duplicated().sum() # 18 duplicated 
# check why the duplicates occur with consumption value
consumption.loc[consumption["Date"].duplicated()]
consumption.loc[consumption["Date"]=="2020125"] # The values within the day 1 to 29 are quite low in differences how to handle the duplicates


# consumption dupes:
consumption["Consumption"].duplicated().sum() # 177 days are the same consumption 
consumption.loc[consumption["Consumption"].duplicated()] # check the duplicate values
consumption.loc[consumption["Consumption"].astype("str").str.contains("245.43")] # validate why they are


# findings:
# 2 types of date formats: dd/mm/yyyy and yyyymmdd
# Dates with the dd/mm/yyyy format tend to usually be part of the 177 consumption duplicates
# Dates with the yyyymmdd are all part of the 18 date duplicates and don't vary too greatly with 50-100 units of consumption variance


,Date,Consumption
498,13/05/2017,245.43
1717,2020913,245.43
2453,19/09/2022,245.43


In [13]:
# Handling the format inconsistencies:
# seperate date into list
dates = consumption["Date"].tolist()

def date_conversion(date_input, conversion_pattern=None):
    """
    A function that converts a string with so far only 2 defined formats into a datetime object.

    Args:
        date_input (str): a date string

    Returns:
        date_time_object : date object of converted string given the 2 types of formats.

    """
    if conversion_pattern == None:
        format1 = "%d/%m/%Y" # equivalent to dd/mm/yyyy
        format2 = "%Y%m%d" # equivalent to yyyymmdd

        try:
            date_obj = datetime.strptime(date_input, format1)
        except:
            date_obj = datetime.strptime(date_input, format2)

        
    else:
        date_obj = datetime.strptime(date_input, conversion_pattern)

    return date_obj

In [15]:
# test function: 
converted_dates = [date_conversion(d) for d in dates]

years = [d.year for d in converted_dates]

unique_years = list(set(years))
len(unique_years) # the function worked and there are 7 unique years that can be assigned a value

7

In [17]:
# add conversion to dataframe:

consumption["Date"] = [date_conversion(i) for i in consumption["Date"]]
consumption["Year"] = [i.strftime("%Y") for i in consumption["Date"]]
consumption["Month"] = [i.strftime("%m") for i in consumption["Date"]]
consumption["index"] = [i.strftime("%m-%d") for i in consumption["Date"]]
consumption.set_index("index", inplace=True)

cleaned_consumption_df = consumption.drop("Date", axis=1)[["Year", "Consumption", "Month"]]
# Dealing with duplicates:
len(cleaned_consumption_df)
len(cleaned_consumption_df.loc[cleaned_consumption_df.duplicated()])
cleaned_consumption_df.loc[cleaned_consumption_df.duplicated()]

TypeError: strptime() argument 1 must be str, not Timestamp

In [18]:
# mapping month to season based on UK:
def season_mapping(month):
    season_dict = { "Winter": ["12","01","02"],
                    "Spring": ["03", "04", "05"],
                    "Summer": ["06", "07", "08"],
                    "Autumn": ["09", "10", "11"]

    }
    for k,v in season_dict.items():
        if month in v:
            return k
# applying map to dataframe
cleaned_consumption_df["Season"]= [season_mapping(month) for month in cleaned_consumption_df["Month"]]

cleaned_consumption_df

,Year,Consumption,Month,Season
index,,,,
01-01,2016,276.2910,01,Winter
01-02,2016,294.6780,01,Winter
01-03,2016,306.3096,01,Winter
01-04,2016,370.0566,01,Winter
01-05,2016,367.3998,01,Winter
...,...,...,...,...
09-29,2022,258.0822,09,Autumn
09-30,2022,242.5950,09,Autumn
10-01,2022,188.0658,10,Autumn


In [ ]:
# everything needs to be graphed now how they ask
graphing_df = cleaned_consumption_df.reset_index()

five_year_scope = (graphing_df.loc[graphing_df["Year"].astype("int")<=2022].copy())# stop original dataframe from being overwritten
five_year_agg_by_month = five_year_scope.groupby(["Year", "Month"]).mean().reset_index() # the duplicated data has been aggregated out

select_season = alt.selection_multi()

alt.Chart(five_year_agg_by_month).mark_bar().encode(
    alt.X("Month:O"),
    alt.Y("Consumption:Q")
).interactive()

## Question 3 (COMPLETE)

A robot moves around a flat surface from position (0,0). It accepts instructions, and moves accordingly. For example:
“BEGIN”
LEFT 3
UP 5
RIGHT 4
DOWN 7
The first word indicates direction and the number shows steps. The robot will stop moving with instruction “STOP”.
Please write a function, which accepts instructions as a list. When first “STOP” instruction is given, it calculates the distance of Robot from the original position (0,0)

In [20]:
def robot(move):
    command_count = len(move)

    instructions = ["begin", "right", "left", "up", "down", "stop"]

    posUp = 0
    posDown = 0
    posLeft = 0
    posRight = 0
    
    
    for i in range(0, command_count):
        if (move[i].lower() not in instructions):
                print(f"The robot is confused {move[i]} is not a valid command! please re-enter commands that look like this: {instructions}")
                break
        else:

            if (move[i].lower() == "begin"):
                posUp = posDown = posLeft = posRight = 0
                print(f"The robot is starting from the begining position {posRight-posLeft,posUp-posDown}")
            elif (move[i].lower() == "right"):
                posRight +=1
                print(f"The robot moved right one step, current positon is: {posRight-posLeft,posUp-posDown}")
                time.sleep(.5)
            elif (move[i].lower() == "left"):
                posLeft +=1
                print(f"The robot moved left one step current positon is: {posRight-posLeft,posUp-posDown}")
                time.sleep(.5)
            elif (move[i].lower() == "up"):
                posUp +=1
                print(f"The robot moved up one step, current positon is: {posRight-posLeft,posUp-posDown}")
                time.sleep(.5)
            elif (move[i].lower() == "down"):
                posDown +=1
                print(f"The robot moved down one step, current positon is: {posRight-posLeft,posUp-posDown}")
                time.sleep(.5)
            elif (move[i].lower()== "stop"):
                print(f"The robot has stopped! current positon is: {posRight-posLeft,posUp-posDown}")
                break

In [22]:
# Test robot:

moves = ["Begin", "Left", "Left", "Left", "Up", "Up", "Up", "Up", "Up", "Right", "Right", "Right", "Right", "Down", "Down", "Down", "Down", "Down", "Down", "Down", "STOP" ]
robot(moves)

The robot is starting from the begining position (0, 0)
The robot moved left one step current positon is: (-1, 0)
The robot moved left one step current positon is: (-2, 0)
The robot moved left one step current positon is: (-3, 0)
The robot moved up one step, current positon is: (-3, 1)
The robot moved up one step, current positon is: (-3, 2)
The robot moved up one step, current positon is: (-3, 3)
The robot moved up one step, current positon is: (-3, 4)
The robot moved up one step, current positon is: (-3, 5)
The robot moved right one step, current positon is: (-2, 5)
The robot moved right one step, current positon is: (-1, 5)
The robot moved right one step, current positon is: (0, 5)
The robot moved right one step, current positon is: (1, 5)
The robot moved down one step, current positon is: (1, 4)
The robot moved down one step, current positon is: (1, 3)
The robot moved down one step, current positon is: (1, 2)
The robot moved down one step, current positon is: (1, 1)
The robot moved

# Question 4 

You are provided with sample trades data in a data file (trades.csv). It contains a collection of trade data during 5 working days with volume and price. You are asked to process the sample data into a data set for creating the candlestick plots, with open, high, low & close prices (OHLC) and total volume for each product and contract, over a time interval.
You are expected to write a function to create a dataframe contains OHLC and trading volume with (begin, end, product(s), freq) as input. It should be able take freq input, e.g., 15MIN, 1H, 1D.
Expected Result – A python model file detailing process with comments: Note:
• The product “Emission - Venue A” and “Emission - Venue B” are the same product trading in two different venues, please combine them when queried
• If product list has more than one contract, please generate OHLC and volume data for each contract
• Please limit output within trading hours, i.e., 7:00 – 17:00, except when freq >= 1D
• Please do not use OHLC resampler

In [ ]:
trades = pd.read_csv("./Data analyst coding challenge/Trades.csv")
trades.info()
# ensure date time is formated correctly:
trades["TradeDateTime"]= pd.to_datetime(trades["TradeDateTime"], infer_datetime_format=True)

# trades.loc[trades["TradeDateTime"].astype("str").str.contains("09:00:00")]


# for index, row in trades.iterrows():
#     date = row["TradeDateTime"]:

# split date and time
trades["Date"] = [x.strftime("%Y-%m-%d") for x in trades["TradeDateTime"]]
trades["Time"] = [x.strftime("%H:%M:%S") for x in trades["TradeDateTime"]]

# establish open close: # 7:00 17:00 Assumption: the market closes at 5pm on the dot so take the minute before closing as close price
def open_close(t, open_time="07:00:00", close_time="16:59:00"):

    if str(t) == open_time:
        return True
    elif str(t) == close_time:
        return False
    else:
        return "-"

trades["is_open"]= [open_close(i) for i in trades["Time"]]
trades = trades.set_index("TradeDateTime")